# Basics: overfitting a MLP on CIFAR10

# CIFAR-10 dataset

In this notebook, we use the CIFAR-10 dataset made of images from 10 classes: plane, car, bird, cat, deer, dog, frog, horse, ship, and truck. The images are all of the same size with $3\times 32\times 32$ pixels. The first channel is used for the RGB description of the image with the 2 others are the X and Y position of a pixel.

1. What is the machine learning family this problem is about? What is the nature of the input data?

2. Enumerate some methods that could be used for such a task.

3. We separate the dataset into two parts: train and test. Remind what they are used for.

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as t
import matplotlib.pyplot as plt
import numpy as np

# load data
to_tensor =  t.ToTensor()
normalize = t.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
flatten =  t.Lambda(lambda x:x.view(-1))

transform_list = t.Compose([to_tensor, normalize, flatten])
train_set = torchvision.datasets.CIFAR10(root='.', train=True, transform=transform_list, download=True)
test_set = torchvision.datasets.CIFAR10(root='.', train=False, transform=transform_list, download=True)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Plot some images
nx = 6
ny = 6
fig, ax = plt.subplots(nx, ny, figsize=(10,10))
for i in range(nx):
  for j in range(ny):
    ax[i][j].imshow(train_set.data[i*nx+j])
    ax[i][j].axis('off')
    ax[i][j].set_title(classes[train_set.targets[i*nx+j]],
                       fontsize=14)

In [ ]:
# Your code to check the proportions of images in each class

# Training loop

The basic network is a small neural network made of one layer with 1000 neurons.

4. Draw the diagram of the networks declared in the code below. How many parameters does it have?

The model is specified for you. Now you have to train it on the training data using the backpropagation algorithm and stochastic gradient descent.

5. Remind how SGD works.

6. Specify a loss function and an optimization procedure. In our case, we'll use the [cross-entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) together with the [SGD optimizer](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html). Set the learning rate to 0.01 and the momentum to 0.9.


Train the neural network. To compute and update the gradient, there are four steps:
> * First compute the loss on the batch,
> * Clean your optimizer using the [`optimizer.zero_grad()`](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html) function,
> * Call the [`backward()`](https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html) function of the loss to compute the gradient,
> * Make a step in the opposite gradient direction using the [`optimizer.step()`](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html) function.

In [ ]:
# define network structure
net = nn.Sequential(nn.Linear(3 * 32 * 32, 1000), nn.ReLU(), nn.Linear(1000, 10))
criterion = # ---> LOSS FUNCTION
optimizer = # ---> OPTIMIZER

# === Train === ###
net.train()

# train loop
for epoch in range(3):
    train_correct = 0
    train_loss = 0
    print('Epoch {}'.format(epoch))

    # loop per epoch
    for i, (batch, targets) in enumerate(train_loader):

        # ---> Train your network with the help of the text above

        # ---> Your code to get the prediction as 'pred'
        train_correct += pred.eq(targets.view_as(pred)).sum().item()
        train_loss += loss

        if i % 100 == 10: print('Train loss {:.4f}, Train accuracy {:.2f}%'.format(
            train_loss / (i * 64), 100 * train_correct / (i * 64)))

print('End of training.\n')

# === Test === ###
test_correct = 0
net.eval()

# loop, over whole test set
for i, (batch, targets) in enumerate(test_loader):

    # ---> Your code to get the prediction as 'pred'
    test_correct += pred.eq(targets.view_as(pred)).sum().item()

print('End of testing. Test accuracy {:.2f}%'.format(
    100 * test_correct / (len(test_loader) * 64)))

Training loop over CIFAR10 (40,000 train images, 10,000 test images). What happens if you
- switch the training to a GPU? Is it faster?
- Remove the `ReLU()`?
- Increase the learning rate?
- Stack more layers?
- Perform more epochs?

The goal is to completely overfit the training set (i.e. get 100% accuracy?)

This code is highly non-modulable. Create functions for each specific task.
(hint: see [this](https://github.com/pytorch/examples/blob/master/mnist/main.py))

Your training went well. Good. Why not save the weights of the network (`net.state_dict()`) using `torch.save()`?

7. Create a function `train` and `test`.

In [ ]:
# Your code of the functions train and test

8. Train a model on the CPU and one on on the GPU. Compare the running times.

In [ ]:
# Your code to compare CPU and GPU

9. Remove the ReLU activation. What becomes the network in this case? Compute and comments the performances.

In [ ]:
# Your code to compare the ReLU and no ReLU networks

10. Change the learning rate. Comment.

In [ ]:
# Your code to compare learning rates

11. Add more layers. Comment.

In [ ]:
# Your code to add more layers to the network

12. Try to make the network completely fit the training set. What can you do for that? Try it!

In [ ]:
# Your code to make the network overfits completely